# Fast Reads from Azure SQL

This notebook shows how to read data from Azure SQL as fast as possibile

Define variables used thoughout the script. Azure Key Value has been used to securely store sensitive data. More info here: [Create an Azure Key Vault-backed secret scope](https://docs.microsoft.com/en-us/azure/databricks/security/secrets/secret-scopes#--create-an-azure-key-vault-backed-secret-scope)

In [3]:
val scope = "key-vault-secrets"

val server = dbutils.secrets.get(scope, "srv001")
val database = "ApacheSpark"

val jdbcUrl = s"jdbc:sqlserver://$server.database.windows.net;database=$database;"

val connectionProperties = new java.util.Properties()
connectionProperties.put("user", dbutils.secrets.get(scope, "dbuser001"))
connectionProperties.put("password", dbutils.secrets.get(scope, "dbpwd001"))
connectionProperties.setProperty("Driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver")

scope: String = key-vault-secrets
server: String = [REDACTED]
database: String = ApacheSpark
jdbcUrl: String = jdbc:sqlserver://[REDACTED].database.windows.net;database=ApacheSpark;
connectionProperties: java.util.Properties = {user=[REDACTED], password=[REDACTED], Driver=com.microsoft.sqlserver.jdbc.SQLServerDriver}
res0: Object = null

Read data using the most basic options

In [5]:
val li = spark.read.jdbc(jdbcUrl, "dbo.LINEITEM", connectionProperties)

li: org.apache.spark.sql.DataFrame = [L_ORDERKEY: int, L_PARTKEY: int ... 15 more fields]

The `where` predicate can be pushed to Azure SQL automatically

In [7]:
li.select("L_PARTKEY", "L_SUPPKEY").where("L_ORDERKEY=7628996").show()

+---------+---------+
L_PARTKEY|L_SUPPKEY|
+---------+---------+
 256603| 6608|
 154832| 4835|
 1912159| 87217|
 721642| 46650|
 1647932| 22981|
 625449| 50456|
+---------+---------+

In [8]:
li.select("L_PARTKEY", "L_SUPPKEY").where("L_ORDERKEY=7628996").explain()

== Physical Plan ==
*(1) Project [L_PARTKEY#25, L_SUPPKEY#26]
+- *(1) Scan JDBCRelation(dbo.LINEITEM) [numPartitions=1] [L_PARTKEY#25,L_SUPPKEY#26] PushedFilters: [*IsNotNull(L_ORDERKEY), *EqualTo(L_ORDERKEY,7628996)], ReadSchema: struct<L_PARTKEY:int,L_SUPPKEY:int>

But only the `where` clause has this ability. The following query will *not* take advtange of Azure SQL ability to aggregate data

In [10]:
li.select("L_SUPPKEY", "L_EXTENDEDPRICE", "L_PARTKEY").where("L_SUPPKEY=5652").groupBy("L_PARTKEY").avg("L_EXTENDEDPRICE").explain()

== Physical Plan ==
*(2) HashAggregate(keys=[L_PARTKEY#25], functions=[finalmerge_avg(merge sum#83, count#84L) AS avg(L_EXTENDEDPRICE#29)#76])
+- Exchange hashpartitioning(L_PARTKEY#25, 200), [id=#92]
 +- *(1) HashAggregate(keys=[L_PARTKEY#25], functions=[partial_avg(L_EXTENDEDPRICE#29) AS (sum#83, count#84L)])
 +- *(1) Project [L_EXTENDEDPRICE#29, L_PARTKEY#25]
 +- *(1) Scan JDBCRelation(dbo.LINEITEM) [numPartitions=1] [L_EXTENDEDPRICE#29,L_PARTKEY#25] PushedFilters: [*IsNotNull(L_SUPPKEY), *EqualTo(L_SUPPKEY,5652)], ReadSchema: struct<L_EXTENDEDPRICE:decimal(15,2),L_PARTKEY:int>

In [11]:
li.select("L_SUPPKEY", "L_EXTENDEDPRICE", "L_PARTKEY").where("L_SUPPKEY=5652").groupBy("L_PARTKEY").avg("L_EXTENDEDPRICE").show()

+---------+--------------------+
L_PARTKEY|avg(L_EXTENDEDPRICE)|
+---------+--------------------+
 405651| 36401.193846|
 255647| 51284.160000|
 155649| 37502.080000|
 5651| 42548.433333|
 1255627| 36240.624000|
 1905651| 51905.546667|
 1630603| 32203.920000|
 1080641| 47836.905000|
 1005651| 36774.675000|
 130648| 34641.025455|
 555641| 48104.167059|
 1230615| 43584.510000|
 1030621| 43020.804545|
 1505651| 36720.856667|
 55651| 35346.300000|
 1305651| 39758.160000|
 430639| 41071.461667|
 730630| 36533.200000|
 355645| 59279.102857|
 855635| 47717.700000|
+---------+--------------------+
only showing top 20 rows

This is the equivalent query that we could have wrote and executed directly on Azure SQL, improving performance a lot, as only the result would have been transferred and not the whole `LINEITEM` table

```
SELECT
	AVG(L_EXTENDEDPRICE),
	L_PARTKEY
FROM
	dbo.LINEITEM
GROUP BY
	L_PARTKEY
ORDER BY
	L_PARTKEY
```

In [13]:
li.select("L_EXTENDEDPRICE", "L_PARTKEY").groupBy("L_PARTKEY").avg("L_EXTENDEDPRICE").orderBy("L_PARTKEY").show()

+---------+--------------------+
L_PARTKEY|avg(L_EXTENDEDPRICE)|
+---------+--------------------+
 1| 24575.551724|
 2| 26655.655172|
 3| 24276.807692|
 4| 20043.862069|
 5| 23406.590909|
 6| 19497.120000|
 7| 23868.421053|
 8| 20029.411765|
 9| 22854.857143|
 10| 24050.264286|
 11| 24354.334000|
 12| 23958.749189|
 13| 23703.144231|
 14| 23078.752500|
 15| 23668.258667|
 16| 23277.430588|
 17| 23763.659143|
 18| 24021.261667|
 19| 20811.129677|
 20| 26828.970323|
+---------+--------------------+
only showing top 20 rows

All the above limits are applicable also when using Spark SQL

In [15]:
li.createOrReplaceTempView("LINEITEM");

In [16]:
%sql

SELECT
	AVG(L_EXTENDEDPRICE),
	L_PARTKEY
FROM
	`LINEITEM`
GROUP BY
	L_PARTKEY
ORDER BY
	L_PARTKEY

## Real World Sample

Let's now simulate a real world scenario and see what can be done to optimize it. The following reference query *MUST* be executed on Spark as Azure SQL doesn't support yet `interval` windows. So all data must be transferred to Spark, and so we need to do it as fast as possibile

In [18]:
%sql
WITH cte1 AS
(
  SELECT
    datediff(L_RECEIPTDATE, L_SHIPDATE) as ShipTime,
    L_COMMITDATE
  FROM
    `LINEITEM`
), 
cte2 AS
(
  SELECT    
    AVG(ShipTime) AS AvgTimeToReceive,
    L_COMMITDATE
  FROM
    cte1
  GROUP BY
    L_COMMITDATE
)
SELECT
  *,
  AVG(AvgTimeToReceive) OVER (ORDER BY CAST(L_COMMITDATE AS TIMESTAMP) RANGE BETWEEN INTERVAL '7' Days PRECEDING AND CURRENT ROW) as 7DaysMovingAvg
FROM
  cte2

AvgTimeToReceive,L_COMMITDATE,7DaysMovingAvg
15.781553398058252,1992-01-31,15.781553398058252
15.336996336996338,1992-02-01,15.559274867527295
15.358851674641148,1992-02-02,15.492467136565246
15.626739261947973,1992-02-03,15.526035167910928
15.767751479289942,1992-02-04,15.574378430186732
15.722947368421053,1992-02-05,15.599139919892451
15.196160558464223,1992-02-06,15.54157143968842
15.332217827806511,1992-02-07,15.515402238203182
15.397201291711518,1992-02-08,15.467358224909841
15.805806294218103,1992-02-09,15.52595946956256


Performance of above query is bad as the table is read using just one thread. To improve performance we can read it in parallel. As table is partitioned, we can read each partition in parallel, up to a desired maximum. 

Read a table in parallel is really fast, as long as you are using the partition column or the clustered index column (in this latter case Azure SQL table do not need to be partitioned to get the performance benefit)

In [20]:
Let's create the partitions:

In [21]:
val ms = 1 to 12;
val ys = 1992 to 1998;

val p = ys.map(y => ms.map(m => y * 100 + m)).flatten.map(pk => s"L_PARTITION_KEY = ${pk.toString}")

ms: scala.collection.immutable.Range.Inclusive = Range(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12)
ys: scala.collection.immutable.Range.Inclusive = Range(1992, 1993, 1994, 1995, 1996, 1997, 1998)
p: scala.collection.immutable.IndexedSeq[String] = Vector(L_PARTITION_KEY = 199201, L_PARTITION_KEY = 199202, L_PARTITION_KEY = 199203, L_PARTITION_KEY = 199204, L_PARTITION_KEY = 199205, L_PARTITION_KEY = 199206, L_PARTITION_KEY = 199207, L_PARTITION_KEY = 199208, L_PARTITION_KEY = 199209, L_PARTITION_KEY = 199210, L_PARTITION_KEY = 199211, L_PARTITION_KEY = 199212, L_PARTITION_KEY = 199301, L_PARTITION_KEY = 199302, L_PARTITION_KEY = 199303, L_PARTITION_KEY = 199304, L_PARTITION_KEY = 199305, L_PARTITION_KEY = 199306, L_PARTITION_KEY = 199307, L_PARTITION_KEY = 199308, L_PARTITION_KEY = 199309, L_PARTITION_KEY = 199310, L_PARTITION_KEY = 199311, L_PARTITION_KEY = 199312, L_PARTITION_KEY = 199401, L_PARTITION_KEY = 199402, L_PARTITION_KEY = 199403, L_PARTITION_KEY = 199404, L_PARTITION_KEY = 199405, L_PARTITION_KEY = 199406, L_PARTITION_KEY = 199407, L_PARTITION_KEY = 199408, L_PARTITION_KEY = 199409, L_PARTITION_KEY = 199410, L_PARTITION_KEY = 199411, L_PARTITION_KEY = 199412, L_PARTITION_KEY = 199501, L_PARTITION_KEY = 199502, L_PARTITION_KEY = 199503, L_PARTITION_KEY = 199504, L_PARTITION_KEY = 199505, L_PARTITION_KEY = 199506, L_PARTITION_KEY = 199507, L_PARTITION_KEY = 199508, L_PARTITION_KEY = 199509, L_PARTITION_KEY = 199510, L_PARTITION_KEY = 199511, L_PARTITION_KEY = 199512, L_PARTITION_KEY = 199601, L_PARTITION_KEY = 199602, L_PARTITION_KEY = 199603, L_PARTITION_KEY = 199604, L_PARTITION_KEY = 199605, L_PARTITION_KEY = 199606, L_PARTITION_KEY = 199607, L_PARTITION_KEY = 199608, L_PARTITION_KEY = 199609, L_PARTITION_KEY = 199610, L_PARTITION_KEY = 199611, L_PARTITION_KEY = 199612, L_PARTITION_KEY = 199701, L_PARTITION_KEY = 199702, L_PARTITION_KEY = 199703, L_PARTITION_KEY = 199704, L_PARTITION_KEY = 199705, L_PARTITION_KEY = 199706, L_PARTITION_KEY = 199707, L_PARTITION_KEY = 199708, L_PARTITION_KEY = 199709, L_PARTITION_KEY = 199710, L_PARTITION_KEY = 199711, L_PARTITION_KEY = 199712, L_PARTITION_KEY = 199801, L_PARTITION_KEY = 199802, L_PARTITION_KEY = 199803, L_PARTITION_KEY = 199804, L_PARTITION_KEY = 199805, L_PARTITION_KEY = 199806, L_PARTITION_KEY = 199807, L_PARTITION_KEY = 199808, L_PARTITION_KEY = 199809, L_PARTITION_KEY = 199810, L_PARTITION_KEY = 199811, L_PARTITION_KEY = 199812)

And now read the table using the defined partitions

In [23]:
val li2 = spark.read.option("numPartitions", 16).jdbc(jdbcUrl, "dbo.LINEITEM", p.toArray, connectionProperties)
li2.createOrReplaceTempView("LINEITEM2");

li2: org.apache.spark.sql.DataFrame = [L_ORDERKEY: int, L_PARTKEY: int ... 15 more fields]

Let's test the performance of this approach

In [25]:
%sql
WITH cte1 AS
(
  SELECT
    datediff(L_RECEIPTDATE, L_SHIPDATE) as ShipTime,
    L_COMMITDATE
  FROM
    `LINEITEM2`
), 
cte2 AS
(
  SELECT    
    AVG(ShipTime) AS AvgTimeToReceive,
    L_COMMITDATE
  FROM
    cte1
  GROUP BY
    L_COMMITDATE
)
SELECT
  *,
  AVG(AvgTimeToReceive) OVER (ORDER BY CAST(L_COMMITDATE AS TIMESTAMP) RANGE BETWEEN INTERVAL '7' Days PRECEDING AND CURRENT ROW) as 7DaysMovingAvg
FROM
  cte2

AvgTimeToReceive,L_COMMITDATE,7DaysMovingAvg
15.781553398058252,1992-01-31,15.781553398058252
15.336996336996338,1992-02-01,15.559274867527295
15.358851674641148,1992-02-02,15.492467136565246
15.626739261947973,1992-02-03,15.526035167910928
15.767751479289942,1992-02-04,15.574378430186732
15.722947368421053,1992-02-05,15.599139919892451
15.196160558464223,1992-02-06,15.54157143968842
15.332217827806511,1992-02-07,15.515402238203182
15.397201291711518,1992-02-08,15.467358224909841
15.805806294218103,1992-02-09,15.52595946956256


Another option is also to use the `$partition` system function

In [27]:
val pn = 1 to 100

val p = pn.map(i => s"$$partition.pf_LINEITEM([L_PARTITION_KEY]) = $i")

pn: scala.collection.immutable.Range.Inclusive = Range(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100)
p: scala.collection.immutable.IndexedSeq[String] = Vector($partition.pf_LINEITEM([L_PARTITION_KEY]) = 1, $partition.pf_LINEITEM([L_PARTITION_KEY]) = 2, $partition.pf_LINEITEM([L_PARTITION_KEY]) = 3, $partition.pf_LINEITEM([L_PARTITION_KEY]) = 4, $partition.pf_LINEITEM([L_PARTITION_KEY]) = 5, $partition.pf_LINEITEM([L_PARTITION_KEY]) = 6, $partition.pf_LINEITEM([L_PARTITION_KEY]) = 7, $partition.pf_LINEITEM([L_PARTITION_KEY]) = 8, $partition.pf_LINEITEM([L_PARTITION_KEY]) = 9, $partition.pf_LINEITEM([L_PARTITION_KEY]) = 10, $partition.pf_LINEITEM([L_PARTITION_KEY]) = 11, $partition.pf_LINEITEM([L_PARTITION_KEY]) = 12, $partition.pf_LINEITEM([L_PARTITION_KEY]) = 13, $partition.pf_LINEITEM([L_PARTITION_KEY]) = 14, $partition.pf_LINEITEM([L_PARTITION_KEY]) = 15, $partition.pf_LINEITEM([L_PARTITION_KEY]) = 16, $partition.pf_LINEITEM([L_PARTITION_KEY]) = 17, $partition.pf_LINEITEM([L_PARTITION_KEY]) = 18, $partition.pf_LINEITEM([L_PARTITION_KEY]) = 19, $partition.pf_LINEITEM([L_PARTITION_KEY]) = 20, $partition.pf_LINEITEM([L_PARTITION_KEY]) = 21, $partition.pf_LINEITEM([L_PARTITION_KEY]) = 22, $partition.pf_LINEITEM([L_PARTITION_KEY]) = 23, $partition.pf_LINEITEM([L_PARTITION_KEY]) = 24, $partition.pf_LINEITEM([L_PARTITION_KEY]) = 25, $partition.pf_LINEITEM([L_PARTITION_KEY]) = 26, $partition.pf_LINEITEM([L_PARTITION_KEY]) = 27, $partition.pf_LINEITEM([L_PARTITION_KEY]) = 28, $partition.pf_LINEITEM([L_PARTITION_KEY]) = 29, $partition.pf_LINEITEM([L_PARTITION_KEY]) = 30, $partition.pf_LINEITEM([L_PARTITION_KEY]) = 31, $partition.pf_LINEITEM([L_PARTITION_KEY]) = 32, $partition.pf_LINEITEM([L_PARTITION_KEY]) = 33, $partition.pf_LINEITEM([L_PARTITION_KEY]) = 34, $partition.pf_LINEITEM([L_PARTITION_KEY]) = 35, $partition.pf_LINEITEM([L_PARTITION_KEY]) = 36, $partition.pf_LINEITEM([L_PARTITION_KEY]) = 37, $partition.pf_LINEITEM([L_PARTITION_KEY]) = 38, $partition.pf_LINEITEM([L_PARTITION_KEY]) = 39, $partition.pf_LINEITEM([L_PARTITION_KEY]) = 40, $partition.pf_LINEITEM([L_PARTITION_KEY]) = 41, $partition.pf_LINEITEM([L_PARTITION_KEY]) = 42, $partition.pf_LINEITEM([L_PARTITION_KEY]) = 43, $partition.pf_LINEITEM([L_PARTITION_KEY]) = 44, $partition.pf_LINEITEM([L_PARTITION_KEY]) = 45, $partition.pf_LINEITEM([L_PARTITION_KEY]) = 46, $partition.pf_LINEITEM([L_PARTITION_KEY]) = 47, $partition.pf_LINEITEM([L_PARTITION_KEY]) = 48, $partition.pf_LINEITEM([L_PARTITION_KEY]) = 49, $partition.pf_LINEITEM([L_PARTITION_KEY]) = 50, $partition.pf_LINEITEM([L_PARTITION_KEY]) = 51, $partition.pf_LINEITEM([L_PARTITION_KEY]) = 52, $partition.pf_LINEITEM([L_PARTITION_KEY]) = 53, $partition.pf_LINEITEM([L_PARTITION_KEY]) = 54, $partition.pf_LINEITEM([L_PARTITION_KEY]) = 55, $partition.pf_LINEITEM([L_PARTITION_KEY]) = 56, $partition.pf_LINEITEM([L_PARTITION_KEY]) = 57, $partition.pf_LINEITEM([L_PARTITION_KEY]) = 58, $partition.pf_LINEITEM([L_PARTITION_KEY]) = 59, $partition.pf_LINEITEM([L_PARTITION_KEY]) = 60, $partition.pf_LINEITEM([L_PARTITION_KEY]) = 61, $partition.pf_LINEITEM([L_PARTITION_KEY]) = 62, $partition.pf_LINEITEM([L_PARTITION_KEY]) = 63, $partition.pf_LINEITEM([L_PARTITION_KEY]) = 64, $partition.pf_LINEITEM([L_PARTITION_KEY]) = 65, $partition.pf_LINEITEM([L_PARTITION_KEY]) = 66, $partition.pf_LINEITEM([L_PARTITION_KEY]) = 67, $partition.pf_LINEITEM([L_PARTITION_KEY]) = 68, $partition.pf_LINEITEM([L_PARTITION_KEY]) = 69, $partition.pf_LINEITEM([L_PARTITION_KEY]) = 70, $partition.pf_LINEITEM([L_PARTITION_KEY]) = 71, $partition.pf_LINEITEM([L_PARTITION_KEY]) = 72, $partition.pf_LINEITEM([L_PARTITION_KEY]) = 73, 

In [28]:
val li2 = spark.read.option("numPartitions", 16).jdbc(jdbcUrl, "dbo.LINEITEM", p.toArray, connectionProperties)
li2.createOrReplaceTempView("LINEITEM2b");

li2: org.apache.spark.sql.DataFrame = [L_ORDERKEY: int, L_PARTKEY: int ... 15 more fields]

In [29]:
%sql
WITH cte1 AS
(
  SELECT
    datediff(L_RECEIPTDATE, L_SHIPDATE) as ShipTime,
    L_COMMITDATE
  FROM
    `LINEITEM2b`
), 
cte2 AS
(
  SELECT    
    AVG(ShipTime) AS AvgTimeToReceive,
    L_COMMITDATE
  FROM
    cte1
  GROUP BY
    L_COMMITDATE
)
SELECT
  *,
  AVG(AvgTimeToReceive) OVER (ORDER BY CAST(L_COMMITDATE AS TIMESTAMP) RANGE BETWEEN INTERVAL '7' Days PRECEDING AND CURRENT ROW) as 7DaysMovingAvg
FROM
  cte2

AvgTimeToReceive,L_COMMITDATE,7DaysMovingAvg
15.781553398058252,1992-01-31,15.781553398058252
15.336996336996338,1992-02-01,15.559274867527295
15.358851674641148,1992-02-02,15.492467136565246
15.626739261947973,1992-02-03,15.526035167910928
15.767751479289942,1992-02-04,15.574378430186732
15.722947368421053,1992-02-05,15.599139919892451
15.196160558464223,1992-02-06,15.54157143968842
15.332217827806511,1992-02-07,15.515402238203182
15.397201291711518,1992-02-08,15.467358224909841
15.805806294218103,1992-02-09,15.52595946956256


If a range partition strategy cannot be used, you can try to take advantage of existing clustered index and ask Spark read the table in parallel, using manually generated buckets. This works well if data is evenly distributed. 

In case clustered index is not there, at least parallel read of table is achieved, but at the expense of a lot of resources used on Azure SQL side, as all queries will do a full table scans.

In [31]:
val li3 = spark.read
  .option("numPartitions", 16)
  .option("partitionColumn", "L_ORDERKEY")
  .option("lowerBound", 1)
  .option("upperBound", 60000000)
  .jdbc(jdbcUrl, "dbo.LINEITEM", connectionProperties)
li3.createOrReplaceTempView("LINEITEM3");

li3: org.apache.spark.sql.DataFrame = [L_ORDERKEY: int, L_PARTKEY: int ... 15 more fields]

Let's test the performance of this approach

In [33]:
%sql
WITH cte1 AS
(
  SELECT
    datediff(L_RECEIPTDATE, L_SHIPDATE) as ShipTime,
    L_COMMITDATE
  FROM
    `LINEITEM3`
), 
cte2 AS
(
  SELECT    
    AVG(ShipTime) AS AvgTimeToReceive,
    L_COMMITDATE
  FROM
    cte1
  GROUP BY
    L_COMMITDATE
)
SELECT
  *,
  AVG(AvgTimeToReceive) OVER (ORDER BY CAST(L_COMMITDATE AS TIMESTAMP) RANGE BETWEEN INTERVAL '7' Days PRECEDING AND CURRENT ROW) as 7DaysMovingAvg
FROM
  cte2

AvgTimeToReceive,L_COMMITDATE,7DaysMovingAvg
15.781553398058252,1992-01-31,15.781553398058252
15.336996336996338,1992-02-01,15.559274867527295
15.358851674641148,1992-02-02,15.492467136565246
15.626739261947973,1992-02-03,15.526035167910928
15.767751479289942,1992-02-04,15.574378430186732
15.722947368421053,1992-02-05,15.599139919892451
15.196160558464223,1992-02-06,15.54157143968842
15.332217827806511,1992-02-07,15.515402238203182
15.397201291711518,1992-02-08,15.467358224909841
15.805806294218103,1992-02-09,15.52595946956256


If a clustered index is available, then performance are much better,and Azure SQL CPU usage is limited

In [35]:
val li4 = spark.read
  .option("numPartitions", 16)
  .option("partitionColumn", "L_ORDERKEY")
  .option("lowerBound", 1)
  .option("upperBound", 60000000)
  .jdbc(jdbcUrl, "dbo.LINEITEM_NONPARTITIONED", connectionProperties)
li4.createOrReplaceTempView("LINEITEM4");

li4: org.apache.spark.sql.DataFrame = [L_ORDERKEY: int, L_PARTKEY: int ... 15 more fields]

In [36]:
%sql
WITH cte1 AS
(
  SELECT
    datediff(L_RECEIPTDATE, L_SHIPDATE) as ShipTime,
    L_COMMITDATE
  FROM
    `LINEITEM4`
), 
cte2 AS
(
  SELECT    
    AVG(ShipTime) AS AvgTimeToReceive,
    L_COMMITDATE
  FROM
    cte1
  GROUP BY
    L_COMMITDATE
)
SELECT
  *,
  AVG(AvgTimeToReceive) OVER (ORDER BY CAST(L_COMMITDATE AS TIMESTAMP) RANGE BETWEEN INTERVAL '7' Days PRECEDING AND CURRENT ROW) as 7DaysMovingAvg
FROM
  cte2

AvgTimeToReceive,L_COMMITDATE,7DaysMovingAvg
15.781553398058252,1992-01-31,15.781553398058252
15.336996336996338,1992-02-01,15.559274867527295
15.358851674641148,1992-02-02,15.492467136565246
15.626739261947973,1992-02-03,15.526035167910928
15.767751479289942,1992-02-04,15.574378430186732
15.722947368421053,1992-02-05,15.599139919892451
15.196160558464223,1992-02-06,15.54157143968842
15.332217827806511,1992-02-07,15.515402238203182
15.397201291711518,1992-02-08,15.467358224909841
15.805806294218103,1992-02-09,15.52595946956256
